# Contributing code and models

- Step 1: **Read the [contribution guide](https://ntc-rosetta.readthedocs.io/en/latest/CONTRIBUTING.html)**!
- Step 2: See Step 1
- Step 3: Check the [support matrix](https://ntc-rosetta.readthedocs.io/en/latest/models/matrix/index.html).

### Goals of this guide
- A *high level* overview of
  * Adding a YANG library
  * Adding models
  * Adding drivers

### Non-goals
- [How to open a PR](https://github.com/networktocode/ntc-rosetta/pulls)
- [How to use Rosetta](https://ntc-rosetta.readthedocs.io/en/latest/tutorials/index.html)
- [How to YANG](https://en.wikipedia.org/wiki/YANG)

## Add a new library or model

The network automation realm is in a constant state of flux, and YANG models are no different with revisions and additions.  First, please double check if the library model you want to support already exists in the [support matrix](https://ntc-rosetta.readthedocs.io/en/latest/models/matrix/index.html).

### Add a library
- Create the appropriate directory structure under `ntc_rosetta/ntc_rosetta/yang`.
- Be sure to include your [YANG library](https://tools.ietf.org/html/rfc8525) json.
- Update the datamodels in `ntc_rosetta/ntc_rosetta/yang/__init__.py`
  - Update the _DATAMODELS hash
  - Create a `get_my_new_library` method and add it to `get_data_model`.

### Add a model
- Put your file in the appropriate directory under the `ntc_rosetta/ntc_rosetta/yang/` folder.
- Update the [YANG library](https://tools.ietf.org/html/rfc8525) json.

## Add a new driver

- Create a file named after your driver in `ntc_rosetta/ntc_rosetta/drivers/my_awesome_driver.py`
  - This file will import the parsers and translators for each YANG library it supports.  See the [existing drivers](https://github.com/networktocode/ntc-rosetta/tree/develop/ntc_rosetta/drivers) for examples.
    - Each YANG library will have its own driver class in the driver file.  The library-specific parsers and translators are registered to its respective class. 
- Import the new driver in `ntc_rosetta/ntc_rosetta/__init__.py` and register it in the `mapping` dict under `get_driver`.

### Creating parsers and translators

This is the directory layout used within the `ntc_rosetta` module.
```bash
ntc_rosetta/{parsers,translators}/{yang_library_name}
├── __init__.py  # <- probably empty
└── {driver_name}
   ├── __init__.py  # <- Needs to import the parsers/translators from each model
   └── {model_name}
      ├── __init__.py  # <- probably empty
      └── {attrib}.py  # <- Implements the Yangify parser/translator for {model_name}
```
- Create the appropriate file structure and write your parsers and translators.  Again, see the [source code](https://github.com/networktocode/ntc-rosetta/tree/develop/ntc_rosetta) for examples.
- As mentioned earlier, each library gets its own class under the master driver, so each `{library_name}Parser` (or Translator) class needs to be registered with [`ntc_rosetta/ntc_rosetta/parsers/__init__.py`](https://github.com/networktocode/ntc-rosetta/blob/develop/ntc_rosetta/parsers/__init__.py).